# 第十六课 图片特征和图像分类

- [ ] 载入clip模型并且抽取图片特征
- [ ] 载入一个文件夹的图片进行抽取
- [ ] 准备两个文件夹的图片
- [ ] 训练分类器
- [ ] 对新的图片进行分类测试
- [ ] 搭建gradio的demo

在之前的课程实践里我们做过一个很有趣的例子，就是准备两个文件夹的照片

然后训练一个图像分类器，就可以对图像进行分类了。让我们在这里实践这个例子。

在计算机视觉课程中我们已经写过一个记录图片和对应标签的代码，

让我们复习一下

------

{粘贴之前任意一个带手检测的程序}

这段程序可以正常运行，我希望修改这段程序

当按下任意非'q'键时，程序会在hand_record_data/hand_data.csv中(续写）记录手的所有关键点的坐标

同时第一列记录按键对应的具体字幕

同时把图片保存在hand_record_data/imgs中，命名规则为按键对应的具体字幕_时间戳.jpg

----

In [5]:
import cv2
import mediapipe as mp
import os
import csv
import time

# 初始化MediaPipe手部模型
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=2,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

# 获取绘图工具
mp_drawing = mp.solutions.drawing_utils

# 打开摄像头
cap = cv2.VideoCapture(0)

# 确保存储目录存在
if not os.path.exists('hand_record_data/hand_data.csv'):
    os.makedirs('hand_record_data/imgs', exist_ok=True)
    with open('hand_record_data/hand_data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        header = ['Key', 'Timestamp']
        for i in range(21):  # Assuming max 21 landmarks per hand
            header.extend([f'x{i}', f'y{i}', f'z{i}'])
        writer.writerow(header)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        continue

    # 左右翻转图像
    frame = cv2.flip(frame, 1)

    # 转换图像颜色空间从BGR到RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    org_frame = frame.copy()

    # 处理图像，检测手部
    results = hands.process(frame_rgb)

    # 绘制手部关键点
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # 显示图像
    cv2.imshow('MediaPipe Hands with Mirror Image', frame)

    # 检测按键输入
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
    elif key != -1:  # 任意其他有效按键
        character = chr(key).lower()
        timestamp = int(time.time())
        image_name = f"{character}_{timestamp}.jpg"
        image_path = f"hand_record_data/imgs/{image_name}"
        cv2.imwrite(image_path, org_frame)  # 保存图像

        # 记录数据
        data = [character, timestamp]
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    data.extend([lm.x, lm.y, lm.z])
        # 保存到CSV文件
        with open('hand_record_data/hand_data.csv', 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)

# 释放资源
hands.close()
cap.release()
cv2.destroyAllWindows()


同学们可以自己分别按下1，2，3来录制不同手势（比如石头、剪刀、布）的数据

这个时候图片和imgs会分别保存到hand_record_data文件夹中

这里李鲁鲁老师已经录制了一部分数据，同学们也可以下载hand_record_data.zip的压缩包进行解压



在对应的imgs文件夹中就会有大量的石头剪刀布的图片

这里石头的图片是以1_时间戳.jpg的形式命名的，以此类推，剪刀是2，布是3

In [2]:
import os
os.environ['HTTP_PROXY'] = 'http://localhost:8234'
os.environ['HTTPS_PROXY'] = 'http://localhost:8234'

在上节课的时候，我们使用实现了src下面的ImageDatabase类

我们可以用ImageDatabase(folder_name) 的方式，来初始化把一个文件夹下面的jpg文件都抽取成特征

这个时候我们有两个办法来处理hand_record_data/imgs文件夹下面的图片

- 修改ImageDatabase类，让它能够处理特性如1_*.jpg开头的图片
- 不修改ImageDatabase类，然后手动把1_*.jpg开头的图片都复制到另一个文件夹，比如叫1_imgs，然后调用ImageDatabase('1_imgs')来处理

这里我们使用第二个方法，当然我们让ChatGPT来帮我们实现这一点

```
在hand_record_data/imgs文件夹下，有很多 1_xxx.jpg, 2_xxx.jpg, 3_xxx.jpg的图片

我希望把1_xxx.jpg的图片都复制到hand_record_data/imgs_1文件夹下（新建这个文件夹）

以此类推也处理2_xxx.jpg, 3_xxx.jpg的图片到对应的文件夹，请帮我实现一段python代码来处理这事情
```

In [7]:
import os
import shutil

def organize_images(src_folder, dst_folder_base):
    # 获取源文件夹中的所有文件
    files = os.listdir(src_folder)
    
    # 遍历文件，根据文件名进行分类
    for file in files:
        if file.endswith('.jpg'):
            # 提取文件名中的数字部分，即分类的编号
            category = file.split('_')[0]
            # 构建目标文件夹路径
            dst_folder = os.path.join(dst_folder_base, f'imgs_{category}')
            # 如果目标文件夹不存在，则创建
            if not os.path.exists(dst_folder):
                os.makedirs(dst_folder)
            # 复制文件到目标文件夹
            shutil.copy(os.path.join(src_folder, file), os.path.join(dst_folder, file))

# 源文件夹和目标文件夹的基础路径
src_folder = 'hand_record_data/imgs'
dst_folder_base = 'hand_record_data'

# 调用函数执行操作
organize_images(src_folder, dst_folder_base)


可以看到运行完这段代码后，图片就被成功归类到三个文件夹了。这个时候我们可以调用ImageDatabase类来处理各个文件夹

In [8]:
from src.ImageDatabase import ImageDatabase

features = []

for i in range(1, 4):
    db = ImageDatabase(f'hand_record_data/imgs_{i}')
    feature = db.get_all_features()
    del db
    # 删除db是为了防止占用过多内存
    features.append(feature)
    print("抽取文件夹{}的特征完成".format(i))

c:\Program Files\Python38\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


抽取文件夹1的特征完成


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


抽取文件夹2的特征完成


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


抽取文件夹3的特征完成


In [7]:
print(features[0].shape)
print(features[1].shape)
print(features[2].shape)

(16, 768)
(14, 768)
(15, 768)


这里我们发现0，1，2都是三个numpy数组

所以我们可以要求ChatGPT帮我们实现一个分类器，并且定义一个predict(feature, model)函数

features是一个list of numpy数组

```python
print(features[0].shape)
print(features[1].shape)
print(features[2].shape)
```

可以发现他们的shape分别为

(16, 768)
(14, 768)
(15, 768)

我希望实现一个随机森林的分类器，对features进行分类，把这个模型保存在hand_record_data/model.pkl

并帮我实现对应的load函数，以及实现一个predict(feature, model) 函数，用来判断一个新的特征是哪个类别

In [9]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pickle
import os

# 假设 features 数据结构如下
# features = [
#     np.random.rand(16, 768),  # 示例数据
#     np.random.rand(14, 768),  # 示例数据
#     np.random.rand(15, 768)   # 示例数据
# ]

# 这里一定要注释掉，不然features被覆盖了！

# 创建目标数组
num_samples = [f.shape[0] for f in features]
classes = np.concatenate([[i] * num_samples[i] for i in range(len(num_samples))])

# 将 features 转换为一个单一的大数组
features_concatenated = np.concatenate(features, axis=0)

# 初始化并训练随机森林分类器
rf_classifier = RandomForestClassifier()
rf_classifier.fit(features_concatenated, classes)

# 保存模型
model_path = 'hand_record_data/model.pkl'
if not os.path.exists('hand_record_data'):
    os.makedirs('hand_record_data')
with open(model_path, 'wb') as file:
    pickle.dump(rf_classifier, file)

# 实现加载模型的函数
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
    return model

# 实现预测函数
def predict(feature, model):
    return model.predict([feature])[0]

# 输出信息
print("模型已保存并实现了加载和预测函数。")


模型已保存并实现了加载和预测函数。


现在我们就可以构建自己的分类器了

完整的从图片到类别的过程要经历特征的抽取，然后再调用predict函数。

In [15]:
from src.CLIPExtractor import CLIPExtractor
cache_dir = "D:\\aistudio\\LubaoGithub\\models"
clip_extractor = CLIPExtractor(model_name = "openai/clip-vit-large-patch14", cache_dir = cache_dir)

# 保存模型
model_path = 'hand_record_data/model.pkl'

# 实现加载模型的函数
def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
    return model

model = load_model(model_path)

example_image = "images/hand_test.jpg"

feature = clip_extractor.extract_image_from_file(example_image)

prediction = predict(feature, model)

texts = ["石头", "剪刀", "布"]

print("预测结果为：", prediction, texts[prediction ])

c:\Program Files\Python38\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


预测结果为： 1 剪刀


我们让ChatGPT为我们这段代码来建立一个gradio的demo

---

{复制上面的代码}

这段代码可以顺利运行

我希望使用with gr.Blocks() as demo:的方式

建立一个gradio的demo，这个gradio会接受一张图片（图片也可以从摄像头捕获）

然后运行分类给出类别的判断

---

In [21]:
import gradio as gr
import pickle
from src.CLIPExtractor import CLIPExtractor

# Initialize CLIP extractor
cache_dir = "D:\\aistudio\\LubaoGithub\\models"
clip_extractor = CLIPExtractor(model_name="openai/clip-vit-large-patch14", cache_dir=cache_dir)

# Load pre-trained model
model_path = 'hand_record_data/model.pkl'

def load_model(path):
    with open(path, 'rb') as file:
        model = pickle.load(file)
    return model

model = load_model(model_path)


# 实现预测函数
def predict(feature, model):
    return model.predict([feature])[0]

# Prediction function
def predict_image(image):
    # Extract features using CLIP extractor
    feature = clip_extractor.extract_image(image)

    # Make a prediction using the loaded model
    prediction = model.predict(feature.reshape(1, -1))  # Assuming model is sklearn-based and accepts feature vector
    texts = ["石头", "剪刀", "布"]
    
    return texts[prediction[0]]

# Set up Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Hand Gesture Recognition Demo")
    
    with gr.Row():
        with gr.Column():
            # image_input = gr.Image(source="upload", tool="editor", type="filepath", label="Upload or Capture an Image")
            # webcam_input = gr.Image(source="webcam", tool="editor", type="filepath", label="Capture from Webcam")
            image_input = gr.Image(label="上传图片")

        with gr.Column():
            output_text = gr.Textbox(label="Prediction", interactive=False)
    
    # Define action on button click
    classify_button = gr.Button("Classify Image")
    classify_button.click(fn=predict_image, inputs=[image_input], outputs=[output_text])

# Launch Gradio app
demo.launch()


c:\Program Files\Python38\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


运行出来的效果就是这样的

![Hand Classification Example](images/hand_classification.jpg)

## 课堂练习

- 寻找互联网上两类图片 比如鸡和鸭 进行分类的尝试
- 其实训练过程（指定文件夹进行训练） 也可以用gradio来实现
- 在这里我们尝试以深度学习模型的特征来进行分类，在计算机视觉课上我们说过手势点也可以作为特征，尝试用手的骨架识别点来作为特征进行分类尝试